Original setting

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pwd
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/M6/week5")
!ls
!pwd

/content
pre_trained.ipynb
/content/drive/My Drive/Colab Notebooks/M6/week5


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Install detectron2

In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 26kB/s 
     |████████████████████████████████| 4.1MB 6.3MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
     |████████████████████████████████| 276kB 45.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=7e04661c372512bb938710e5238c94193b1630b40ef441cfb92704a50adc48f8
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-7ike_rah
  Running command git 

In [0]:
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
     |████████████████████████████████| 6.2MB 2.1MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.dev200325-cp36-none-any.whl size=38930 sha256=1b3667af68665d0f43657ed2ac9041e9c54e6f6314749e6986e2ce65513db83e
  Stored in directory: /root/.cache/pip/wheels/58/4f/d1/6b0ada294145df5fe73092d9849d27be5b5b720fd8cfdd1b98
Successfully built fvcore


# import

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [0]:
import tensorflow
tensorflow.__version__

'1.15.2'

# Run the model

In [0]:
model_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file(model_name))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_name)
predictor = DefaultPredictor(cfg)


In [0]:
from tqdm import tqdm

dataset_location = "../Datasets/AIC20_track3/train/S03/"
detections_all_camera = {}
for sequence in sorted(os.listdir(dataset_location)):
    print (sequence)
    sequence_location = "{}{}/frames/".format(dataset_location, sequence)
    detections = []
    for frame in tqdm(sorted(os.listdir(sequence_location))):
        frame_filename = "{}{}".format(sequence_location, frame)
        im = cv2.imread(frame_filename)
        outputs = predictor(im)

        # print(outputs)
        classes = outputs["instances"].pred_classes.to("cpu").numpy()
        boxes = outputs["instances"].pred_boxes.to("cpu").tensor.numpy()
        scores = outputs["instances"].scores.to("cpu").numpy()

        car_index = np.where(classes==2)[0]
        if len(car_index) > 0:
            for k in range(car_index.shape[0]):
                frame_id = int(frame.split('.')[0])
                bbox = boxes[car_index[k]]
                score = scores[car_index[k]]
                
                detection = {}
                box_w = bbox[2] - bbox[0]
                box_h = bbox[3] - bbox[1]
                detection['frame'] = frame_id
                detection['left'] = bbox[0]
                detection['top'] = bbox[1]
                detection['width'] = box_w
                detection['height'] = box_h
                detection['confidence'] = score
                detections.append(detection)
    detections_all_camera[sequence] = detections


In [0]:
import pickle
with open("detections_all_camera.pkl", 'wb') as f:
    pickle.dump(detections_all_camera, f)
    f.close()